In [1]:
#import relevent libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
print("Libraries imported successfully")

Libraries imported successfully


In [2]:
#load train and test datasets
train = pd.read_csv("datasets/train.csv")
test = pd.read_csv("datasets/test.csv")

train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
input_nodes = test.shape[1]
num_layer = 3 # number of layers
h1 = 200       # hidden layer 1
h2 = 150        # hidden layer 2
output_nodes = len(train['label'].unique())

In [4]:
print("The size of input is {}.".format(input_nodes))
print("The size of output is {}.".format(output_nodes))

The size of input is 784.
The size of output is 10.


In [5]:
#define sigmoid function
def sigmoid(z):
    g=1/(1+np.exp(-z))
    
    return g

In [6]:
#initialize the required parameters with array of appropriate dimensions
def parameters_initialization(input_nodes, h1, h2, output_nodes):
    
    #weights are initialized in each layer with random values
    W1=np.random.randn(h1, input_nodes)*0.01
    W2=np.random.randn(h2, h1)*0.01
    W3=np.random.randn(output_nodes, h2)*0.01
    
#     #bias are initialized in each layer with random values
#     b1=np.zeros((h1,1))
#     b2=np.zeros((h2,1))
#     b3=np.zeros((output,1))
    
    return W1,W2,W3
    

In [7]:
#
def forward_propagation(input, W1, W2, W3):
    
    #calculate linear equation of input times weight
    Z1=np.dot(W1,input)
    A1=sigmoid(Z1)
    Z2=np.dot(W2,A1)
    A2=sigmoid(Z2)
    Z3=np.dot(W3,A2)
    A3=sigmoid(Z3)
    
    return A3,A2,A1
    

In [8]:
#calculate cost function
def cost_function(output,A3):
    m=42000
    cost=(-1/m)*np.sum(output*np.log(A3)+(1-output*np.log(1-A3)))
    
    return cost


In [20]:
def back_propagation(input,output,learning_rate,A3,A2,A1,W3,W2,W1):
    m=42000
    dZ3=A3-output
    dZ2=np.dot(W3.T,dZ3)*(A2-np.power(A2,2))
    dZ1=np.dot(W2.T,dZ2)*(A1-np.power(A1,2))
    
    W1=W1-learning_rate*(1/m)*np.dot(dZ1,input.T)
    W2=W2-learning_rate*(1/m)*np.dot(dZ2,A1.T)
    W3=W3-learning_rate*(1/m)*np.dot(dZ3,A2.T)
    
#     b1=b1-learning_rate*(1/m)*np.sum(dZ1, axis=1, keepdims=True)
#     b2=b2-learning_rate*(1/m)*np.sum(dZ2, axis=1, keepdims=True)
#     b3=b3-learning_rate*(1/m)*np.sum(dZ3, axis=1, keepdims=True)
    
    return W1,W2,W3 
    
    

In [24]:
input=np.transpose(np.asfarray(train.iloc[:,1:]))/255.0
learning_rate=0.001

output=[]
for i in range(42000):
    out =np.zeros(10)+0.01
    out[train['label'][i]]=0.99
    output.append(out)
output=np.transpose(output)
    
print("Input:")
print(input)
print("Output:")
print(output)


Input:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Output:
[[0.01 0.99 0.01 ... 0.01 0.01 0.01]
 [0.99 0.01 0.99 ... 0.01 0.01 0.01]
 [0.01 0.01 0.01 ... 0.01 0.01 0.01]
 ...
 [0.01 0.01 0.01 ... 0.99 0.01 0.01]
 [0.01 0.01 0.01 ... 0.01 0.01 0.01]
 [0.01 0.01 0.01 ... 0.01 0.01 0.99]]


In [25]:
#train the neural network
epochs=25
for epoch in range(epochs):
    print("Epoch: "+ str(epoch+1)+"/"+str(10))
    W1, W2, W3= parameters_initialization(input_nodes, h1, h2, output_nodes)
    A3,A2,A1 = forward_propagation(input, W1, W2, W3)
    cost=cost_function(output,A3)
    print(cost)
    W1,W2,W3=back_propagation(input,output,learning_rate,A3,A2,A1,W3,W2,W1)
    
    

    
    

Epoch: 1/10
-10.012203671640837
Epoch: 2/10
-10.02619323203153
Epoch: 3/10
-9.997715020699788
Epoch: 4/10
-10.040526505942664
Epoch: 5/10
-9.944859628600842
Epoch: 6/10
-10.030078077599375
Epoch: 7/10
-10.008744433451296
Epoch: 8/10
-9.950084142730894
Epoch: 9/10
-10.009354162804337
Epoch: 10/10
-9.981033431370781
Epoch: 11/10
-9.971174291836492
Epoch: 12/10
-9.998088751558573
Epoch: 13/10
-9.962689338829186
Epoch: 14/10
-10.028438885527631
Epoch: 15/10
-10.00394279587228
Epoch: 16/10
-10.0319746100621
Epoch: 17/10
-10.017317522730362
Epoch: 18/10
-10.032357777427153
Epoch: 19/10
-10.03722435260573
Epoch: 20/10
-10.049182852690201
Epoch: 21/10
-10.027565010570514
Epoch: 22/10
-9.969431407657176
Epoch: 23/10
-10.0465430318973
Epoch: 24/10
-10.013586315879083
Epoch: 25/10
-9.974325313410382


In [26]:
#predict from the test data

input_test=np.transpose(np.asfarray(test))/255.0
A3=forward_propagation(input_test, W1, W2, W3)
A3=np.transpose(A3)
print(A3)



[array([[0.48150828, 0.48150365, 0.48150418, ..., 0.48149506, 0.48146124,
        0.48151268],
       [0.49247553, 0.49241807, 0.49249147, ..., 0.49250641, 0.49245966,
        0.49246854],
       [0.48188074, 0.48190123, 0.48187458, ..., 0.48188177, 0.48189605,
        0.48187206],
       ...,
       [0.50352201, 0.50353456, 0.50350662, ..., 0.50353032, 0.50353528,
        0.50351889],
       [0.49726297, 0.49724309, 0.49728961, ..., 0.49729957, 0.49728757,
        0.4972948 ],
       [0.49489334, 0.49488544, 0.4948915 , ..., 0.4949191 , 0.49488823,
        0.49486639]])
 array([[0.48826019, 0.48832696, 0.487834  , ..., 0.48653999, 0.48627099,
        0.48789128],
       [0.51332288, 0.51436213, 0.51238526, ..., 0.51378456, 0.51359217,
        0.51269927],
       [0.51278405, 0.51300376, 0.51306087, ..., 0.51184428, 0.51346877,
        0.51405052],
       ...,
       [0.5258317 , 0.52723093, 0.52669604, ..., 0.52694223, 0.52633348,
        0.52580534],
       [0.51820986, 0.51807553, 0